In [1]:
# init settings
import os
import sys
pwd = os.getcwd()
sys.path.append(os.path.join(pwd, '..', '..'))

from server.utils import load_config
from server import db
from sqlalchemy.orm import sessionmaker
import json
import matplotlib.pyplot as plt

conf_dir = os.path.abspath(os.path.join(pwd, '..', '..', 'config', 'base.yaml'))
config = load_config(conf_dir)

engine = db.sync_engine(config['postgres'])
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
# Query data

from sqlalchemy.orm.exc import NoResultFound
import sqlalchemy as sa

rate = False
all_orders = False
start_date_filter = '2017-04-10 00:00'
end_date_filter = '2017-04-13 23:59'

pair = 'btc_usd'
cursor = session.query(db.order).filter(
    (db.order.c.extra['is_finished'].astext == '0')
    & (db.order.c.pair == pair)
    & (db.order.c.pub_date > start_date_filter)
    & (db.order.c.pub_date < end_date_filter)
).order_by(db.order.c.pub_date)

if all_orders:
    all_orders = session.query(db.order).filter(
        (db.order.c.pair == pair)
        & (db.order.c.pub_date > start_date_filter)
        & (db.order.c.pub_date < end_date_filter)
    ).order_by(db.order.c.pub_date)

if rate:
    rate = session.query(db.history).filter(
        (db.history.c.pub_date > start_date_filter)
        & (db.history.c.pub_date < end_date_filter)
        & (db.history.c.pair == pair)
        & (sa.sql.func.random() < 0.0001)
    ).order_by(db.history.c.pub_date)

def as_dict(row):
    item = row._asdict().copy()
    item['api'] = item['api']
    return item

def iter_tail(item, tail):
    next_steps = session.query(db.order).filter(
        ((db.order.c.extra['next'].astext == str(item['id']))
        | (db.order.c.extra['merged'].astext == str(item['id']))
        ) 
        & (db.order.c.pub_date > start_date_filter)
        & (db.order.c.pub_date < end_date_filter)
    )
    if not next_steps.count():
        if len(tail) == 0:
            tail.append([item, item])
        return tail
    else: 
        for next_step in next_steps:
            next_step_item = as_dict(next_step)
            #print(item, next_step_item)
            tail.append([item, next_step_item])
            tail = iter_tail(next_step_item, tail)
        return tail

In [37]:
# Data proccesing
order_dict = {
    'sell': [],
    'buy': []
}
if all_orders:
    for order in all_orders:
        order_dict['sell' if order.is_sell else 'buy'].append({
            'date': order.pub_date,
            'price': order.price
        })
if rate:   
    rate_dict = {
        'date': [],
        'sell_price': [],
        'buy_price': []
    }
    for rate_info in rate:
        resp = json.loads(rate_info.resp)
        rate_dict['date'].append(rate_info.pub_date)
        rate_dict['buy_price'].append(resp['asks'][0][0])
        rate_dict['sell_price'].append(resp['bids'][0][0])
    print(len(rate_dict['date']))
    
thread_list = []

all_btc_summ = 0
all_usd_summ = 0
for index, order in enumerate(cursor):
    #if index in [0,1,2,3]:
    #    continue
    item = as_dict(order)
    tail = []
    tail = iter_tail(item, tail)
    thread_list.append(tail)
    usd_sum = 0
    btc_sum = 0
    print('------ thread items ----')
    for index, (next_order, previous) in enumerate(tail[::-1]):
        if not next_order['api']:
            print('invalid API {}'.format(next_order['id']))
            continue
            
        if not previous['api']:
            print('invalid API {}'.format(previous['id']))
            continue
            
        if (float(previous['api']['received']) < 0.0001):
            print('it is order not deal {} {} {}'.format(
                previous['id'],
                previous['api']['received'],
                previous['pub_date']
            ))
            continue
            
        if (float(next_order['api']['received']) < 0.0001):
            print('it is order not deal {} {} {}'.format(
                next_order['id'], 
                next_order['api']['received'],
                next_order['pub_date']
            ))
            continue
            
        print(
            'previos price {} id {}'.format(previous['price'], previous['id']),
            'sell' if previous['is_sell'] else 'buy',
            'next price {} {}'.format(next_order['price'], next_order['id']),
            'sell' if next_order['is_sell'] else 'buy',
        )
        
        btc_money_spend = float(previous['api']['funds']['btc']) - float(previous['api']['old_balance']['btc'])
        usd_money_spend = float(previous['api']['funds']['usd']) - float(previous['api']['old_balance']['usd'])
        
        btc_sum += btc_money_spend
        usd_sum += usd_money_spend
        
        if previous['is_sell']:
            if previous['extra'].get('merged'):
                print('merged {} {} usd {} btc'.format(previous['id'], usd_money_spend, btc_money_spend))
                continue
            if previous['id'] == next_order['id']:
                print('not working thread {} {} usd {} btc'.format(previous['id'], usd_money_spend, btc_money_spend))
                continue
        else:
            if previous['extra'].get('merged'):
                print('merged {} {} usd {} btc'.format(previous['id'], usd_money_spend, btc_money_spend,))
                continue
            if previous['id'] == next_order['id']:
                print('not working thread {} {} usd {} btc'.format(previous['id'], usd_money_spend, btc_money_spend))
                continue
        
        if index + 1 == len(tail):
            usd_money_spend = float(next_order['api']['funds']['btc']) - float(next_order['api']['old_balance']['btc'])
            btc_money_spend = float(next_order['api']['funds']['usd']) - float(next_order['api']['old_balance']['usd'])
            print('last {} {} usd {} btc'.format(next_order['id'], usd_money_spend, btc_money_spend))
            usd_sum += usd_money_spend
            btc_sum += usd_money_spend
            
        print('{} usd {} btc'.format(usd_money_spend, btc_money_spend))
        
    print('------ thread summ ----')
    print('{} usd {} btc'.format(usd_sum, btc_sum))
    all_usd_summ += usd_sum
    all_btc_summ += btc_sum
    #break
print('Total: {} usd {} btc'.format(all_usd_summ, all_btc_summ))

------ thread items ----
previos price 1206.005 id 5604 buy next price 1206.005 5604 buy
not working thread 5604 -1.2060049999999904 usd 0.0009979999999999989 btc
------ thread summ ----
-1.2060049999999904 usd 0.0009979999999999989 btc
------ thread items ----
previos price 1205.0 id 5556 buy next price 1207.452 5557 sell
-1.2050000000000125 usd 0.0009979999999999989 btc
previos price 1207.452 id 5557 sell next price 1200.01 5605 buy
1.2050371000000126 usd -0.0010000000000000009 btc
previos price 1200.01 id 5605 buy next price 1200.01 5606 buy
merged 5605 -1.200010000000006 usd 0.0009979999999999989 btc
previos price 1206.291 id 5588 buy next price 1207.691 5589 sell
-1.2062909999999931 usd 0.0009979999999999989 btc
previos price 1207.691 id 5589 sell next price 1200.01 5606 buy
1.2052756199999948 usd -0.0010000000000000009 btc
previos price 1200.01 id 5606 buy next price 1200.01 5607 buy
merged 5606 -1.2000099999999918 usd 0.0009979999999999989 btc
previos price 1203.798 id 5598 buy 

In [19]:
# Data visualisite

plt.cla()
plt.clf()
fig = plt.figure(figsize=(20,10))

ay = fig.add_subplot(1,1,1)
ay.grid(True)

for name, vals in order_dict.items():
    ay.plot(
        list(map(lambda i: i['date'], vals)), 
        list(map(lambda i: i['price'], vals)),
        color = 'red' if name == 'sell' else 'blue'
    )

if rate:    
    ay.plot(rate_dict['date'], rate_dict['buy_price'], alpha=0.2, color='blue')
    ay.plot(rate_dict['date'], rate_dict['sell_price'], alpha=0.2, color='red')

if all_orders:
    fig

In [ ]:
plt.cla()
plt.clf()

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
cm = plt.get_cmap('Vega10')
for index, thread in enumerate(thread_list):
    for line in thread:
        ax.plot(
            list(map(lambda i: i['pub_date'], line)), 
            list(map(lambda i: i['price'], line)),
            color=cm(index)
        )
        
if rate:    
    ax.plot(rate_dict['date'], rate_dict['buy_price'], alpha=0.2, color='blue')
    ax.plot(rate_dict['date'], rate_dict['sell_price'], alpha=0.2, color='red')
    
fig